# In place RMS calculation using RDKIT Maximum Common Substructure 

Root Mean Square (RMS) calculation is a frequent method of for comparing 3D spatial distribution of molecules or conformers. However, in some cases we want to calculate the RMS of two molecules in place. On other words, without align the molecules.

In this brief workflow, I will show you how to use RDKIT to do a in place RMS calculation for two conformers that come from docking experiments and have a different pose.

## Importing the libraries we will use

To get the atom mapping of our docking poses we will use the Maximum Common Substructure ([MCS](http://rdkit.org/docs/source/rdkit.Chem.MCS.html)) search module from RDKIT. For comparison with other RMS in place method, I will use pymol Rms_cur method ([Rms_cur](https://pymolwiki.org/index.php/Rms_cur)).

In [1]:
# Libraries I will use for RDKIT approach
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolAlign,rdFMCS,Draw

import itertools
from scipy.spatial import distance
import numpy as np
import math

# Libraries I will use for Pymol approach
from pymol import cmd

## Loading the poses

After docking I have a .sdf file for each molecule with 10 poses. I want to compare all my poses

In [ ]:
file=''

In [ ]:
path='../Untitled_Folder/'
for file in os.listdir(path):
    mols=[]
    if '.sdf' in file:
        print (file.split('.')[0],'\n')
        for mol in Chem.SDMolSupplier(path+file,sanitize=False):
            if mol !=None:
                rdmolops.Cleanup (mol)
                mols.append (mol)
        mols[0].SetProp ('_Name','GOLD')
        mols[1].SetProp ('_Name','VINA')
        mols[2].SetProp ('_Name','FRED')
        
        c=itertools.combinations(mols,2)
        unq=list(set(c))
        for i,j in unq:
            distances=[]
            mc=rdFMCS.FindMCS ([i,j])
            a=i.GetSubstructMatch(Chem.MolFromSmarts(mc.smartsString))
            b=j.GetSubstructMatch(Chem.MolFromSmarts(mc.smartsString))
            amap=list(zip(a,b))
            for atomA, atomB in amap:
                pos_A=i.GetConformer().GetAtomPosition (atomA)
                pos_B=j.GetConformer().GetAtomPosition (atomB)
                coord_A=np.array((pos_A.x,pos_A.y,pos_A.z))
                coord_B=np.array ((pos_B.x,pos_B.y,pos_B.z))
                dist_numpy = np.linalg.norm(coord_A-coord_B)                
                distances.append(dist_numpy)
            
            rmsd=math.sqrt(1/len(distances)*sum([i*i for i in distances]))
            print (i.GetProp('_Name'),'Vs', j.GetProp('_Name'),'RMSD:',rmsd,'\n')